## Códigos após adequação da Base Original e inserção de mais informações

In [ ]:
#Importação das Bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from prophet import Prophet
import sklearn.cluster as cluster
import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: ignored

In [ ]:
df = pd.read_csv('CityTemperatureAjustada.csv')
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
#Escolhemos o EUA como referencia para o projeto
#filtrando 
us_cities = df[df['DimCidade_NomPais'] == 'US']
us_cities.head()

In [ ]:
us_cities = df[df['DimCidade_NomPais'] == 'US']
print('Cidades: ', us_cities['DimCidade_NomCidade'].unique(), 'Total :', us_cities['DimCidade_NomCidade'].nunique())

In [ ]:
#Delimitamos para 20 Cidades

top_20 = ['Atlanta','Boston','Chicago','Denver','Detroit','Honolulu','Houston','Indianapolis',
        'Las Vegas','Los Angeles','Miami Beach','New York City','Oklahoma City','Orlando',
        'Philadelphia','Sacramento','San Diego','San Francisco','Seattle','Washington DC']

#Dados com as cidades Top 20

top_20_us = pd.DataFrame(us_cities[us_cities['DimCidade_NomCidade'].isin(top_20)])
top_20_us.head()

## Mapa com as 20 cidades escolhidas

In [ ]:
# Plotando as cidades no Mapa
#DimCidade_ValLatitude
#DimCidade_ValLongitude
#DimCidade_NomCidade

#Importação dos pacotes
import folium 
import pandas as pd
import plotly.express as px
import requests

#Criando uma tabela somente com as cidades, coordenadas e Cluster para plotar o Mapa
top_20_us_mapa = top_20_us.groupby(['DimCidade_NomCidade'])['DimCidade_NomCidade','DimCidade_ValLatitude', 
                                                            'DimCidade_ValLongitude', 'ClusterCidade_ClusterFinal'].agg('max')


mapa = folium.Map(                                  
    location=[34.797152, -103.677898],
    zoom_start=3.4
)

#Plotando
for _, cidade in top_20_us_mapa.iterrows():
    folium.Marker(
        location=[cidade['DimCidade_ValLatitude'], cidade['DimCidade_ValLongitude']],
    ).add_to(mapa)

mapa

### Mapa com a informação de divisão das cidades por Cluster

In [ ]:
top_20_us_mapa.head(20)

In [ ]:
# Plotando as cidades no Mapa
#DimCidade_ValLatitude
#DimCidade_ValLongitude
#DimCidade_NomCidade
#ClusterCidade_ClusterFinal

mapa2 = folium.Map(                                  
    location=[34.797152, -103.677898],
    zoom_start=3.4
)


#Color code the markers to show blue markers for public universities and brown colors for private universities
for i in range(0,len(top_20_us_mapa)):
    cluster = top_20_us_mapa['ClusterCidade_ClusterFinal'].iloc[i]
    if cluster == 1:
        color = 'blue'
    elif cluster == 2:
        color = 'red'
    elif cluster == 3:
        color = 'orange'
    elif cluster == 4:
        color = 'green'
        
    labels=top_20_us_mapa['ClusterCidade_ClusterFinal'].iloc[i] #Create a lable that is the name of the institution

    folium.Marker([top_20_us_mapa['DimCidade_ValLatitude'].iloc[i],top_20_us_mapa['DimCidade_ValLongitude'].iloc[i]],popup=labels,
                 icon=folium.Icon(color=color, icon='cluster', prefix='fa')).add_to(mapa2) 
mapa2

### Legenda Clusters:
Cluster 1 = Azul;
Cluster 2 = Vermelho;
CLuster 3 = Laranja;
Cluster 4 = Verde.

In [ ]:
# Gráfico da média das temperaturas das 20 Cidades ao longo do tempo

#EDA das 20 Cidades Escolhidas

top_20_us = pd.Series(round(top_20_us.groupby('DimCidade_NomCidade')['FatTempearatura_ValMedioTemperaturaCelsius'].mean().sort_values(),2))
top_20_us.plot(kind = 'bar', figsize = (10,6), alpha = 0.5)
plt.xlabel('Cidade')
plt.ylabel('Temperatura Média')
plt.title('Media por Cidade')

In [ ]:
# Gráficos da média das temperaturas das 20 Cidades por Semana, Semestre e Trimestre

top_20_us = pd.DataFrame(us_cities[us_cities['DimCidade_NomCidade'].isin(top_20)])

#EDA Trimestral
plt.subplot(3,3,2)
top_20_us.groupby(['DimCalendario_DescTrimestre'])['FatTempearatura_ValMedioTemperaturaCelsius'].mean().plot(kind='bar')
plt.xlabel('Trimestre')
plt.ylabel('Temperatura')
plt.title('Media Geral por Trimestre')

#EDA Semestral
plt.subplot(3,3,3)
top_20_us.groupby(['DimCalendario_DescSesmestre'])['FatTempearatura_ValMedioTemperaturaCelsius'].mean().plot(kind='bar')
plt.xlabel('Semestre')
plt.ylabel('Temperatura')
plt.title('Media Geral por Semestre')

#EDA Semanal
plt.subplot(3,3,1) 
top_20_us.groupby(['DimCalendario_NumSemanaAno'])['FatTempearatura_ValMedioTemperaturaCelsius'].mean().plot(kind='bar',figsize=(40,40))
plt.xlabel('Semana')
plt.ylabel('Temperatura')
plt.title('Media Geral por Semana')

In [ ]:
# Gráfico da média das temperaturas das 20 Cidades por estação do Ano

top_20_us = pd.DataFrame(us_cities[us_cities['DimCidade_NomCidade'].isin(top_20)])

#EDA Estações

plt.subplot(2,1,1)
top_20_us.groupby(['DimCalendario_DescEstacao'])['FatTempearatura_ValMedioTemperaturaCelsius'].mean().plot(kind='bar',figsize=(17,7))
plt.xlabel('Estação')
plt.ylabel('Temperatura')
plt.title('Media Geral por Estações do Ano')



In [ ]:
# Gráficos de Scaterplot com informações de temperatura média por cidade, estado e numero do mes


top_20_us = pd.DataFrame(us_cities[us_cities['DimCidade_NomCidade'].isin(top_20)])

sns.pairplot(data=top_20_us[['FatTempearatura_ValMedioTemperaturaCelsius','DimCidade_NomEstado','DimCalendario_NumMes']], hue='DimCidade_NomEstado',height=7, aspect=1)



In [ ]:
# Gráfico com os valores médios das temperaturas das cidades com a informação de Cluster

#EDA Cluster

plt.figure(figsize=(10,8))
plt.title('Media das temperaturas das cidades com Cluster')
sns.scatterplot(x ='FatTempearatura_ValMedioTemperaturaCelsius' , y = 'DimCidade_NomCidade', hue = 'ClusterCidade_ClusterFinal', data = top_20_us)


In [ ]:
# Informações da tabela
top_20_us.head()

In [ ]:
# Informações da tabela
top_20_us.shape

In [ ]:
# Retirando as colunas para adequar a base aos modelos que serão utilizados

top_20_us_new = top_20_us.drop(columns=['DimCalendario_CodSkData',
       'DimCalendario_NumAno', 'DimCalendario_DescAno',
       'DimCalendario_NumSemestre', 'DimCalendario_DescSesmestre',
       'DimCalendario_NumTrimestre', 'DimCalendario_DescTrimestre',
       'DimCalendario_NumMes', 'DimCalendario_DescMes',
       'DimCalendario_NumSemanaAno', 'DimCalendario_DescSemanaAno',
       'DimCalendario_NumDia', 'DimCalendario_NumDiaSemana',
       'DimCalendario_DescDiaSemana', 'DimCalendario_FlgFinalSemana',
       'DimCalendario_DescEstacao', 'DimCidade_CodSkCidade', 'DimCidade_NomEstado', 'DimCidade_NomPais',
       'DimCidade_NomRegiaoGlobo', 'DimCidade_ValLatitude',
       'DimCidade_ValLongitude', 'DimCidade_FlgHemisferioNorte',
       'ClusterCidade_CategoriaInverno', 'ClusterCidade_TempMediaInverno',
       'ClusterCidade_CategoriaOutono', 'ClusterCidade_TempMediaOutuno',
       'ClusterCidade_CategoriaPrimavera', 'ClusterCidade_TempMediaPrimavera',
       'ClusterCidade_CategoriaVerao', 'ClusterCidade_TempMediaVerao',
       'ClusterCidade_ClusterDefinicao','DimCidade_NomCidade'])

top_20_us_new.head()


In [ ]:
#Fazer a divisão dos CLusters da tabela - Divisão dos Clusters

cluster1 = pd.DataFrame(top_20_us_new[top_20_us_new['ClusterCidade_ClusterFinal'] == 1])
cluster2 = pd.DataFrame(top_20_us_new[top_20_us_new['ClusterCidade_ClusterFinal'] == 2])
cluster3 = pd.DataFrame(top_20_us_new[top_20_us_new['ClusterCidade_ClusterFinal'] == 3])
cluster4 = pd.DataFrame(top_20_us_new[top_20_us_new['ClusterCidade_ClusterFinal'] == 4])

#cluster1.head()
#cluster2.head()
#cluster3.head()
#cluster4.head()

#O Algoritimo Prophet necessita apenas de duas colunas, coluna Data e Coluna "Objetivo", no caso a temperatura.
# Necessário retirar mais colunas para adequação ao modelo PROPHET

new_cluster1 = cluster1.drop(columns=['ClusterCidade_ClusterFinal'])
new_cluster2 = cluster2.drop(columns=['ClusterCidade_ClusterFinal'])
new_cluster3 = cluster3.drop(columns=['ClusterCidade_ClusterFinal'])
new_cluster4 = cluster4.drop(columns=['ClusterCidade_ClusterFinal'])




In [ ]:
#Testes para saber se os Datasets dos Clusters, está correto
new_cluster1.head()


In [ ]:
new_cluster2.head()

In [ ]:
new_cluster3.head()

In [ ]:
new_cluster4.head()

In [ ]:
#Prophet tambem solicita que a coluna Data e a coluna Objetivo, estejam escritas como 'ds' e 'y' respectivamente, Colunas renomeadas

new_cluster1 = new_cluster1.rename(columns={'DimCalendario_DataCalendario': 'ds', 'FatTempearatura_ValMedioTemperaturaCelsius': 'y'})
new_cluster2 = new_cluster2.rename(columns={'DimCalendario_DataCalendario': 'ds', 'FatTempearatura_ValMedioTemperaturaCelsius': 'y'})
new_cluster3 = new_cluster3.rename(columns={'DimCalendario_DataCalendario': 'ds', 'FatTempearatura_ValMedioTemperaturaCelsius': 'y'})
new_cluster4 = new_cluster4.rename(columns={'DimCalendario_DataCalendario': 'ds', 'FatTempearatura_ValMedioTemperaturaCelsius': 'y'})

In [ ]:
#Fazendo a verificação

new_cluster1.head()

## Fazer as predições de cada cluster usando o modelo Prophet

## Modelo - Cluster 1

In [ ]:
#Iniciando a predição por clusters

modelo1 = Prophet(interval_width= 0.95)
modelo1 = modelo1.fit(new_cluster1)

In [ ]:
#Fazendo as predições

future1 = modelo1.make_future_dataframe(periods= 1327, freq= 'D')
future1.tail()

In [ ]:
#Predict

forecast1 = modelo1.predict(future1)
forecast1

In [ ]:
# Gráficos Modelo - CLuster 1
fig_cluster1 = modelo1.plot(forecast1)
fig_cluster12= modelo1.plot_components(forecast1)

## Modelo - Cluster 2

In [ ]:
#Fazendo o mesmo procedimento para os outros Clusters
#Cluster 2

modelo2 = Prophet()
modelo2 = modelo2.fit(new_cluster2)



In [ ]:
future2 = modelo2.make_future_dataframe(periods= 1327, freq= 'D')
future2.tail()

In [ ]:
#Predict

forecast2 = modelo2.predict(future2)
forecast2

In [ ]:
# Gráfico Modelo - Cluster 2
fig_cluster2 = modelo2.plot(forecast2)
fig_cluster22= modelo2.plot_components(forecast2)

## Modelo - Cluster 3

In [ ]:
#Cluster 3

modelo3 = Prophet()
modelo3 = modelo3.fit(new_cluster3)


In [ ]:
future3 = modelo3.make_future_dataframe(periods= 1327, freq= 'D')
future3.tail()

In [ ]:
#Predict

forecast3 = modelo3.predict(future3)
forecast3

In [ ]:
# Gráfico Modelo - Cluster 3
fig_cluster3 = modelo3.plot(forecast3)
fig_cluster32= modelo3.plot_components(forecast3)

## Modelo - Cluster 4

In [ ]:
#Cluster 4

modelo4 = Prophet()
modelo4 = modelo4.fit(new_cluster4)


In [ ]:
future4 = modelo3.make_future_dataframe(periods= 1327, freq= 'D')
future4.tail()

In [ ]:
#Predict

forecast4 = modelo4.predict(future4)
forecast4

In [ ]:
# Gráficos Modelo - Cluster 4
fig_cluster4 = modelo4.plot(forecast4)
fig_cluster42= modelo4.plot_components(forecast4)

## Modelo - Sem cluster tabela geral

In [ ]:
top_20_us_new = top_20_us_new.rename(columns={'DimCalendario_DataCalendario': 'ds', 'FatTempearatura_ValMedioTemperaturaCelsius': 'y'})
top_20_us_new



In [ ]:
#model = Prophet()
#model.fit(top_20_us_new)

In [ ]:
#future = model.make_future_dataframe(periods = 1327, freq='D' )
#future.tail()

In [ ]:
#forecast = model.predict(future)
#forecast

In [ ]:
#fig_geral = model.plot(forecast)
#fig_geral2 = model.plot_components(forecast)


In [ ]:
#forecast1.head()

### Após a realização do processo de treinamento do modelo em cada cluster será feito a junção das quatro bases

In [ ]:
# Criando a coluna cluster em cada dataset
forecast1['Cluster'] = 1
forecast2['Cluster'] = 2
forecast3['Cluster'] = 3
forecast4['Cluster'] = 4


In [ ]:
forecast1.head()

In [ ]:
forecast2.head()

### Fazendo o merge das 4 bases

In [ ]:
# Juntando as bases
df_final = pd.merge(forecast1, forecast2, how = 'outer')

df_final2= pd.merge(forecast3,forecast4, how = 'outer')

df_versao_final = pd.merge(df_final, df_final2,  how ='outer')
df_versao_final.tail()

df_basefinal = df_versao_final[['ds', 'yhat', 'Cluster']]
df_basefinal.head()

In [ ]:
df_basefinal.tail()

## Tabela com os dados finais

In [ ]:
# Informações da tabela final
df_basefinal.shape

In [ ]:
# Renomeando as colunas
df_basefinal = df_basefinal.rename(columns={'ds': 'Data', 'yhat': 'Celsius'})
df_basefinal.head()

In [ ]:
# Formatando a coluna Celsius para uma casa decimal após a vírgula
df_basefinal['Celsius'] = round((df_basefinal['Celsius']),1)
df_basefinal.tail()

In [ ]:
# Renomeando as colunas da tabela que contém os dados resumidos de Cidade, Coordenadas e Cluster
data_top20 = top_20_us_mapa.rename(columns={'DimCidade_NomCidade': 'City', 'DimCidade_ValLatitude': 'Lat', 
                                                'DimCidade_ValLongitude': 'Long', 'ClusterCidade_ClusterFinal': 'Cluster'})
data_top20

## CÓDIGO DE SAÍDA / RESULTADO

### Foi feito uma interface para que o usuário podesse inserir os dados de entrada:
- Opção 1: Inserir as datas futuras desejadas e a faixa de temperatura Min e Máx;
- Resultado Opção 1: Lista de cidades que terão a temperatura na faixa escolhida durante o intervalo de datas desejadas.

- Opção 2: Inserir a Cidade desejada e a faixa de temperatura Min e Máx;
- Resultado Opção 2: Informar em que faixa de Data a cidade escolhida terá a faixa de temperatura desejada.

- Opção 3: Inserir a cidade desejada com a faixa de datas futuras escolhida;
- Resultado Opção 3: Informar a temperatura Mínina, Média e Máxima na cidade escolhida durante as datas desejadas.

### Pacotes Utilizados

In [ ]:
import pandas as pd

import PySimpleGUI as psg # Utilizado para gerar o layout final;
from datetime import datetime # Utilizado para tratar datas
from html2image import Html2Image # Utilizado para converter o arquivo HTML para Imagem
import folium # Utilizado para geraçao do Mapa

### Códigos de Entrada, Processamento e Saída/Resultado

In [ ]:
# Usado o PySimpleGui para fazer o layout
import PySimpleGUI as psg 

#ENTRADA

#Coletar os dados de Entrada 

#Escolhendo o tema da janela
psg.theme('SandyBeach')
#Defininação do Layout da janela
layout=[[psg.Text('G12 TURISMO',size=(23, 1), font='Lucida',justification='center')],
        [psg.Text('Insira os dados desejados para sua viagem',size=(45, 1), font='Lucida',justification='left')],
        [psg.Text('Cidade:',size=(23, 1), font='Lucida',justification='left'),
        psg.Combo(top_20,default_value='',key='combo_cidade')],
        
        [psg.Text('Data Inicial', size =(23, 1), font='Lucida',justification='left'), psg.InputText()],
        [psg.Text('Data Final (Máx 2023-12-31)', size =(23, 1), font='Lucida',justification='left'), psg.InputText()],
        [psg.Text('Temperatura Minima', size =(23, 1), font='Lucida',justification='left'), psg.InputText()],
        [psg.Text('Temperatura Máxima', size =(23, 1), font='Lucida',justification='left'), psg.InputText()], 
        [psg.Button('OK', font=('Times New Roman',12)),psg.Button('CANCEL', font=('Times New Roman',12))]]

# Configuração da Janela
win =psg.Window('G12 TURISMO - PREDIÇÃO DE TEMPERATURA',layout)

# Fazer a leitura dos valores dos Inputs
inputs_texto, values=win.read()

#close first window
win.close()

combo_cidade = values['combo_cidade']  # Coletando a informação do Combo
in_cidade=combo_cidade;
in_datainicio=values[0];
in_datafinal=values[1];
in_temperaturamin=values[2];
in_temperaturamax=values[3];

  
    
# RESULTADOS

# De acordo com as entradas mostrar o resultado/saída


#### SAÍDA - CIDADES ####


# Mostrar as cidades que atendem aos requisitos de datas e temperaturas - OK
if in_cidade == "":
    
    #Converter os inputs de Data para Date
    in_datainicio = pd.to_datetime(in_datainicio)
    in_datainicio = in_datainicio.strftime('%Y-%m-%d')
    in_datafinal = pd.to_datetime(in_datafinal)
    in_datafinal = in_datafinal.strftime('%Y-%m-%d')
    
    # Converter os valores de temperatura para Float
    in_temperaturamin = float(in_temperaturamin) # Mudar formato para Float
    in_temperaturamax = float(in_temperaturamax) # Mudar formato para Float
    
    # Localizar as informações 
    resultado = df_basefinal.loc[(df_basefinal["Data"]>= in_datainicio) & (df_basefinal["Data"]<=in_datafinal) 
                               & (df_basefinal["Celsius"]>= in_temperaturamin) & (df_basefinal["Celsius"]<=in_temperaturamax)]
    
    resultado1 = resultado["Cluster"].unique() # Pegar a lista de Cluster da pesquisa
    
    list(resultado1)
    
    
    data_top20S=data_top20['Cluster'].isin(resultado1)
    data_top20 = data_top20[data_top20S]
    data_top20.head()     
   
        
    # Fazer um loop para inserir a lista de cidades no PopUp
    resultado_cidade= ""     
    for cidade in data_top20["City"].unique():            
        resultado_cidade=resultado_cidade+ " "+ cidade+ ", "                
                
    
    #transformar dados para String para exibiçao no PopUp
    #resultado_data_max = repr(resultado_data_max)
    #resultado_data_min = repr(resultado_data_min)
    in_temperaturamin = repr(in_temperaturamin)
    in_temperaturamax = repr(in_temperaturamax)
        
    #### SAÍDA - CIDADES ####
    # Mostrar resultado no PopUp       
    texto_saida = ('Lista de cidades de acordo com as datas e temperaturas escolhidas:' +  
          '\nValores de Entrada:\nNa faixa de temperatura MIN de '+ in_temperaturamin + '°C e MAX de '+ in_temperaturamax + '°C'
          '\n- Data Inicial: '+ in_datainicio +
          '\n- Data Final: '+ in_datafinal +
          '\nResultado Lista das Cidades: '+ resultado_cidade)
 
    #print('Texto Saida Cidades: ',texto_saida)

    

#### SAÍDA - DATAS ####


# Mostrar as datas iniciais e final de acordo com a cidade e faixa de temperatura - OK
if (in_datainicio == "") or (in_datafinal == ""):
    
    # Mostrar datas maiores que o dia atual
    from datetime import datetime
    hoje = datetime.today().strftime('%Y-%m-%d')    
    
    
    # Converter os valores de temperatura para Float
    in_temperaturamin = float(in_temperaturamin) # Mudar formato para Float
    in_temperaturamax = float(in_temperaturamax) # Mudar formato para Float
    
    # Localizar as informações
    
    # Localizar o Cluster de acordo com a cidade escolhida
    cluster = data_top20.loc[(data_top20["City"]==in_cidade)]
    cluster = cluster["Cluster"].unique() 
    cluster = float(cluster) # Mudar formato para Float    
    
    resultado = df_basefinal.loc[(df_basefinal["Celsius"]>= in_temperaturamin) & (df_basefinal["Celsius"]<=in_temperaturamax) 
                               & (df_basefinal["Cluster"]==cluster) & (df_basefinal["Data"]>hoje)]    
           
        
    resultado_data_max = resultado["Data"].max().strftime('%Y-%m-%d')
    resultado_data_min = resultado["Data"].min().strftime('%Y-%m-%d')
    
    #transformar dados para String para exibiçao no PopUp    
    resultado_data_max = repr(resultado_data_max)
    resultado_data_min = repr(resultado_data_min)
    in_temperaturamin = repr(in_temperaturamin)
    in_temperaturamax = repr(in_temperaturamax)
    
    resultado_cidade = "" #Limpando o resultado pra não dar erro na geração do Mapa.
    
    #### SAÍDA - DATAS ####
    # Mostrar resultado no PopUp       
    texto_saida = ('Informação das datas para a cidade e faixa de temperaturas escolhidas: \n' +
            'Valores de Entrada:\nCidade: '+ in_cidade + 
            '\nTemperatura MIN de '+ in_temperaturamin + '°C e MAX de '+ in_temperaturamax + '°C' +          
            '\nResultado:\n- Data Inicial: '+ resultado_data_min +
            '\n- Data Final: '+ resultado_data_max         
         )
   
    #print('Texto Saida Datas: ',texto_saida)   
    
    
    
#### SAÍDA - TEMPERATURA ####
    
# Mostrar temperatura (Pegar MIN, Média e MAX) - OK
if (in_temperaturamin == "") or (in_temperaturamax == ""): 
    
    # Localizar as informações
    
    # Localizar o Cluster de acordo com a cidade escolhida
    cluster = data_top20.loc[(data_top20["City"]==in_cidade)]
    cluster = cluster["Cluster"].unique() 
    cluster = float(cluster) # Mudar formato para Float    
    
    
    resultado = df_basefinal.loc[(df_basefinal["Data"]>= in_datainicio) & (df_basefinal["Data"]<=in_datafinal) 
                               & (df_basefinal["Cluster"]==cluster)]
    
    resultado_temperatura_media = resultado["Celsius"].mean()
    resultado_temperatura_media = round(resultado_temperatura_media,1)
    resultado_temperatura_max = resultado["Celsius"].max()
    resultado_temperatura_min = resultado["Celsius"].min()
    
    
    #transformar dados para String para exibiçao no PopUp    
    resultado_temperatura_media = repr(resultado_temperatura_media)
    resultado_temperatura_max = repr(resultado_temperatura_max)
    resultado_temperatura_min = repr(resultado_temperatura_min)    
    
    resultado_cidade = "" #Limpando o resultado pra não dar erro na geração do Mapa.

    #### SAÍDA - TEMPERATURA ####
    # Mostrar resultado no PopUp       
    texto_saida = ('Valores de temperatura para a cidade e período escolhido: \n' +   
           'Valores de Entrada:\nCidade: ' + in_cidade + '.\nData inicial ' + in_datainicio +  ' até Data Final ' + in_datafinal +
          '\nResultado:\n- Temperatura Minima: ' + resultado_temperatura_min + '°C ' +
          '\n- Temperatura Média: ' + resultado_temperatura_media + '°C ' +
          '\n- Temperatura Máxima: ' + resultado_temperatura_max + '°C ')
         
    #print('Texto Saida Temperaturas: ',texto_saida)
    
    
    

### GERANDO O MAPA COM OS DADOS DE ENTRADA E SAÍDA


# GERANDO O MAPA

#convertendo as colunas Lat e Long para Float
data_top20['Lat'] = data_top20['Lat'].astype(float, errors = 'raise')
data_top20['Long'] = data_top20['Long'].astype(float, errors = 'raise')


#Criando uma tabela somente com as cidades e coordenadas
data_top20_mapa = data_top20.groupby(['City'])['City','Lat', 'Long'].agg('max')


# Verificar se a informação da cidade é do resultado ou da entrada
if in_cidade == "":
    #Selecionar somente a cidade(s) que foram selecionadas na entrada ou saída  
    resultado_cidade1=resultado_cidade.split(',')   # Convertendo para LISTA 
    data_top20_mapa[data_top20_mapa["City"].isin(resultado_cidade1)]   
    mapa_lat = data_top20_mapa['Lat'].mean()
    mapa_long = data_top20_mapa['Long'].mean()    
    mapa_zoom = 3.4    
    
else:
    data_top20_mapa = data_top20_mapa.loc[(data_top20_mapa["City"]==in_cidade)]
    # Se for somente uma cidade dar um zoom na cidade e centralizar o mapa
    mapa_lat = data_top20_mapa['Lat'].unique()    
    mapa_long = data_top20_mapa['Long'].unique()    
    mapa_zoom = 10


# Gerar o layout do Mapa (localização do centro e zoom)
mapa = folium.Map(                                  
    location=[mapa_lat, mapa_long],
    zoom_start=mapa_zoom
)


#Plotando
for _, cidade in data_top20_mapa.iterrows():
    folium.Marker(
        location=[cidade['Lat'], cidade['Long']],
    ).add_to(mapa)

              
        
#Salvar o mapa
mapa.save('mapa.html') #O mapa é salvo no diretório onde esse notebook está salvo

# Converter o arquivo HTML para JPG para carregar no PopUp
from html2image import Html2Image

hti = Html2Image()


css = "body {}"

hti.screenshot(
    html_file='mapa.html', css_str=css,
    save_as='mapa.jpeg'
)


#### SAÍDA - POP UP
import PySimpleGUI as psg
message = ""

def popup(title, filename, message, width=90):

    lines = texto_saida
    height = 8
    message = texto_saida

    layout = [ 
        [psg.Button('Close')],
        [psg.Text(message, size=(width, height), justification='left', expand_x=True)],
        [psg.Image(filename=filename, expand_x=True)],        
    ]

    psg.Window(title, layout, size=(960, 600), keep_on_top=True, modal=True).read(close=2000)

popup('G12 Turismo', 'mapa.jpeg', message)
